# Summary

This file has been taken from various versions of the GEE_pull_functions files that have been used to pull surface reflectance from Landsat Collection 1 over rivers, lakes, wqp matchups, etc. It has been modified to run with Landsat Collcetion 2, based on changes with collection 2 data such as: 

* Bit mask information (i.e. bit numbers) 
* Scaling parameters for optical bands 
* Band names 
* Thresholds (inputs, rather) for the DSWE function


In [2]:
#import geemap
import time
import os
import time
import ee
import os
import numpy as np
import pandas

/var/folders/hr/595wnzq15yd0_45l45wq253h0000gn/T/ipykernel_58825/1049827998.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [3]:
ee.Authenticate()


Successfully saved authorization token.


In [4]:
ee.Initialize()

# Image Masking 

Masks are binary images used to remove unwanted pixels from the final layer. 

Below are the updated bit numbers within the 'QA_PIXEL' band for LS Collection 2 data. Note that this is one of the main differences between collection 1 and 2. 

*    Bit 0: Fill
*    Bit 1: Dilated Cloud
*    Bit 2: Cirrus (high confidence)
*    Bit 3: Cloud
*    Bit 4: Cloud Shadow
*    Bit 5: Snow
*    Bit 6: Clear
*        0: Cloud or Dilated Cloud bits are set
*        1: Cloud and Dilated Cloud bits are not set
*    Bit 7: Water
*    Bits 8-9: Cloud Confidence
*        0: None
*        1: Low
*        2: Medium
*        3: High
*    Bits 10-11: Cloud Shadow Confidence
*        0: None
*        1: Low
*        2: Medium
*        3: High
*    Bits 12-13: Snow/Ice Confidence
*        0: None
*        1: Low
*        2: Medium
*        3: High
*    Bits 14-15: Cirrus Confidence
*        0: None
*        1: Low
*        2: Medium
*        3: High

# Unpack

The following functions 'unpack' the information stored within the QA band. 

In [5]:

def Unpack(bitBand, startingBit, bitWidth): #For reference, the Water bit, bit 7, would have a starting bit of 7 and a bit width of 1
  return (ee.Image(bitBand)\
  .rightShift(startingBit)\
  .bitwiseAnd(ee.Number(2).pow(ee.Number(bitWidth)).subtract(ee.Number(1)).int()))


def UnpackAll(bitBand, bitInfo):
  unpackedImage = ee.Image.cat([Unpack(bitBand, bitInfo[key][0], bitInfo[key][1]).rename([key]) for key in bitInfo])
  return unpackedImage



# Function of Mask (FMask)

Fmask is used for cloud, cloudshadow, snow/ice, and water masking 

In [6]:
def AddFmask(image):
    bitInfo = {
    'Cloud': [3, 1],
    'CloudShadow': [4, 1], 
    'SnowIce': [5, 1],
    'Water': [7, 1]
    }
    
    temp = UnpackAll(image.select(['pixel_qa']), bitInfo)
    
    fmask = (temp.select(['Water']).rename(['fmask'])
    .where(temp.select(['SnowIce']), ee.Image(4)) #4 because we're taking SnowIce bit number (5) and subtracting 1
    .where(temp.select(['CloudShadow']), ee.Image(3))
    .where(temp.select(['Cloud']), ee.Image(2))
    .mask(temp.select(['Cloud']).gte(0))) 
    #mask the fmask so that it has the same footprint as the quality (BQA) band
    return(image.addBands(fmask))

In [7]:
def add_rad_mask(image):
  """Mask out all pixels that are radiometrically saturated using the QA_RADSAT
  QA band.

  Args:
      image: ee.Image of an ee.ImageCollection

  Returns:
      ee.Image with additional band called 'radsat', where pixels with a value 
      of 0 are saturated for at least one SR band and a value of 1 is not saturated
  """
  #grab the radsat band
  satQA = image.select('radsat_qa')
  # all must be non-saturated per pixel
  satMask = satQA.eq(0).rename('radsat')
  return image.addBands(satMask).updateMask(satMask)

In [8]:
def sr_aerosol(image):
  """Flags any pixels in Landsat 8 and 9 that have 'medium' or 'high' aerosol QA flags from the
  SR_QA_AEROSOL band.

  Args:
      image: ee.Image of an ee.ImageCollection

  Returns:
      ee.Image with additional band called 'medHighAero', where pixels are given a value of 1
      if the aerosol QA flag is medium or high and 0 otherwise
  """
  aerosolQA = image.select('aerosol_qa')
  medHighAero = aerosolQA.bitwiseAnd(1 << 7).rename('medHighAero')# pull out mask out where aeorosol is med and high
  return image.addBands(medHighAero)

In [9]:
def sr_cloud_mask(image):
  """Masks any pixles in Landsat 4-7 that are contaminated by the inputs of
  the atmospheric processing steps
  Args:
      image: ee.Image of an ee.ImageCollection
  Returns:
      ee.Image with additional band called 'sr_cloud', where pixels are given values
      based on the SR_CLOUD_QA band informaiton. Generally speaking, 0 is clear, values
      greater than 0 are obstructed by clouds and/or snow/ice specifically from atmospheric
      processing steps
  """
  srCloudQA = image.select('cloud_qa')
  srMask = (srCloudQA.bitwiseAnd(1 << 1).rename('sr_cloud') # cloud
    .where(srCloudQA.bitwiseAnd(1 << 2), ee.Image(2)) # cloud shadow
    .where(srCloudQA.bitwiseAnd(1 << 3), ee.Image(3)) # adjacent to cloud
    .where(srCloudQA.bitwiseAnd(1 << 4), ee.Image(4))) # snow/ice
  return image.addBands(srMask)

In [10]:
def apply_fill_mask(image):
  b1_mask = image.select('SR_B1').gt(0)
  b2_mask = image.select('SR_B2').gt(0)
  b3_mask = image.select('SR_B3').gt(0)
  b4_mask = image.select('SR_B4').gt(0)
  b5_mask = image.select('SR_B5').gt(0)
  b7_mask = image.select('SR_B7').gt(0)
  fill_mask = (b1_mask.eq(1)
    .And(b2_mask.eq(1))
    .And(b3_mask.eq(1))
    .And(b4_mask.eq(1))
    .And(b5_mask.eq(1))
    .And(b7_mask.eq(1)))
  return image.updateMask(fill_mask.eq(1))

# This should be applied AFTER scaling factors
# Mask values less than -0.01
def apply_realistic_mask(image):
  b1_mask = image.select('SR_B1').gt(-0.01)
  b2_mask = image.select('SR_B2').gt(-0.01)
  b3_mask = image.select('SR_B3').gt(-0.01)
  b4_mask = image.select('SR_B4').gt(-0.01)
  b5_mask = image.select('SR_B5').gt(-0.01)
  b7_mask = image.select('SR_B7').gt(-0.01)
  realistic = (b1_mask.eq(1)
    .And(b2_mask.eq(1))
    .And(b3_mask.eq(1))
    .And(b4_mask.eq(1))
    .And(b5_mask.eq(1))
    .And(b7_mask.eq(1)))
  return image.updateMask(realistic.eq(1))

# mask high opacity (>0.3 after scaling) pixels
def apply_opac_mask(image):
  opac = image.select("SR_ATMOS_OPACITY").multiply(0.001).lt(0.3)
  return image.updateMask(opac)


# function to split QA bits
def extract_qa_bits(qa_band, start_bit, end_bit, band_name):
  """
  Extracts specified quality assurance (QA) bits from a QA band. This function originated
  from https://calekochenour.github.io/remote-sensing-textbook/03-beginner/chapter13-data-quality-bitmasks.html

  Args:
      qa_band (ee.Image): The earth engine image QA band to extract the bits from.
      start_bit (int): The start bit of the QA bits to extract.
      end_bit (int): The end bit of the QA bits to extract (not inclusive)
      band_name (str): The name to give to the output band.

  Returns:
      ee.Image: A single band image of the extracted QA bit values.
  """
  # Initialize QA bit string/pattern to check QA band against
  qa_bits = 0
  # Add each specified QA bit flag value/string/pattern to the QA bits to check/extract
  for bit in range(end_bit):
    qa_bits += (1 << bit)
  # Return a single band image of the extracted QA bit values
  return (qa_band
    # Rename output band to specified name
    .select([0], [band_name])
    # Check QA band against specified QA bits to see what QA flag values are set
    .bitwiseAnd(qa_bits)
    # Get value that matches bitmask documentation
    # (0 or 1 for single bit,  0-3 or 0-N for multiple bits)
    .rightShift(start_bit))


In [11]:
def apply_high_aero_mask(image):
  qa_aero = image.select('SR_QA_AEROSOL')
  aero = extract_qa_bits(qa_aero, 6, 8, 'aero_level')
  aero_mask = aero.lt(3)
  return image.updateMask(aero_mask)

# Dynamic Surface Water Extent (DSWE)

These functions are for calculating DSWE. The thresholds for various tests (t1-t5), haven't necessarily changed. What's changed are the inputs. Previously, for LS Collection 1 the inputs for these functions were unscaled surface reflectance. Now, the inputs are scaled surface reflectance. 


In [12]:

# Modified Normalized Difference Water Index (MNDWI) 
def Mndwi(image): 
  return(image
  .expression('(GREEN - SWIR1) / (GREEN + SWIR1)', {
    'GREEN': image.select(['Green']),
    'SWIR1': image.select(['Swir1'])
  }))

# Multi-band Spectral Relationship Visible
def Mbsrv(image):
  return(image.select(['Green']).add(image.select(['Red'])).rename('mbsrv'))

# Multi-band Spectral Relationship Near infrared
def Mbsrn(image):
  return(image.select(['Nir']).add(image.select(['Swir1'])).rename('mbsrn'))

# Normalized Difference Vegetation Index
def Ndvi(image):
  return(image
  .expression('(NIR - RED) / (NIR + RED)', {
    'RED': image.select(['Red']),
    'NIR': image.select(['Nir'])
  }))

# Automated Water Extent Shadow
def Awesh(image):
  return(image
  .expression('Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2', {
    'Blue': image.select(['Blue']),
    'Green': image.select(['Green']),
    'mbsrn': Mbsrn(image).select(['mbsrn']),
    'Swir2': image.select(['Swir2'])
  }))

def Dswe(i):
   mndwi = Mndwi(i)
   mbsrv = Mbsrv(i)
   mbsrn = Mbsrn(i)
   awesh = Awesh(i)
   swir1 = i.select(['Swir1'])
   nir = i.select(['Nir'])
   ndvi = Ndvi(i)
   blue = i.select(['Blue'])
   swir2 = i.select(['Swir2'])
   green = i.select(['Green'])
   red = i.select(['Red'])
  
  # These thresholds are taken from the LS Collection 2 DSWE Data Format Control Book:
  # (https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-2042_LandsatC2_L3_DSWE_DFCB-v2.pdf)
  # Inputs are meant to be scaled reflectance values 

   t1 = mndwi.gt(0.124) # MNDWI greater than Wetness Index Threshold
   t2 = mbsrv.gt(mbsrn) # MBSRV greater than MBSRN
   t3 = awesh.gt(0) #AWESH greater than 0
   t4 = (mndwi.gt(-0.44)  #Partial Surface Water 1 thresholds
   .And(swir1.lt(0.09)) #900 for no scaling (LS Collection 1)
   .And(nir.lt(0.15)) #1500 for no scaling (LS Collection 1)
   .And(ndvi.lt(0.7))) 
   t5 = (mndwi.gt(-0.5) #Partial Surface Water 2 thresholds
   .And(blue.lt(0.1)) #1000 for no scaling (LS Collection 1)
   .And(swir1.lt(0.3)) #3000 for no scaling (LS Collection 1)
   .And(swir2.lt(0.1)) #1000 for no scaling (LS Collection 1)
   .And(nir.lt(0.25))) #2500 for no scaling (LS Collection 1)
  
   # Add additional threshold to account for algal blooms which can get masked out 

   algal_mask = green.gt(0.05).And(red.lt(0.04))

   t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000));

   noWater = (t.eq(0)
   .Or(t.eq(1))
   .Or(t.eq(10))
   .Or(t.eq(100))
   .Or(t.eq(1000)))
   hWater = (t.eq(1111)
   .Or(t.eq(10111))
   .Or(t.eq(11011))
   .Or(t.eq(11101))
   .Or(t.eq(11110))
   .Or(t.eq(11111)))
   mWater = (t.eq(111)
   .Or(t.eq(1011))
   .Or(t.eq(1101))
   .Or(t.eq(1110))
   .Or(t.eq(10011))
   .Or(t.eq(10101))
   .Or(t.eq(10110))
   .Or(t.eq(11001))
   .Or(t.eq(11010))
   .Or(t.eq(11100)))
   pWetland = t.eq(11000)
   lWater = (t.eq(11)
   .Or(t.eq(101))
   .Or(t.eq(110))
   .Or(t.eq(1001))
   .Or(t.eq(1010))
   .Or(t.eq(1100))
   .Or(t.eq(10000))
   .Or(t.eq(10001))
   .Or(t.eq(10010))
   .Or(t.eq(10100)))
  
   iDswe = (noWater.multiply(0)
   .add(hWater.multiply(1))
   .add(mWater.multiply(2))
   .add(pWetland.multiply(3))
   .add(lWater.multiply(4)))

  # Reclassify the dswe values so that we combine the lower confidence dswe values together 
   dswe23 = iDswe.remap([0,1,2,3,4], [0,0,1,1,1])
   
   alg = algal_mask.eq(1)
   d_algae = dswe23.updateMask(alg)
   algae = d_algae.eq(1)
   dswe1 = iDswe.eq(1)
   
   # Define pixels that are high confidence water (dswe1_threshold) and pixels that fall within our algal mask threshold (alg_selfmask)
   alg_selfmask = algae.selfMask()
   dswe1_selfmask = dswe1.selfMask()
   
   # Multiply both classes by 1 to combine into an image 
   classes = ee.Image([alg_selfmask, dswe1_selfmask]).multiply(ee.Image([1])).reduce(ee.Reducer.firstNonNull()).unmask(0)
  
   return i.addBands(iDswe.rename('dswe')).addBands(classes.rename('algal_mask'))

# Hillshades and hillshadows

These functions calculate hillshade and hillhshadow based on information on azimuth and zenith angles stored within a Landsat image. 

The names for the azimuth angle was changed from 'SOLAR_AZIMUTH' to 'SUN_AZIMUTH'. There also is no 'ZENITH' field, but a 'SUN_ELEVATION' field that you can do 90 - 'SUN_ELEVATION' to get 'SUN_ZENITH'. 

In [13]:
def CalcHillShades(image, geo):

  MergedDEM = ee.Image("users/eeProject/MERIT").clip(geo.buffer(300)) # Area buffered that hillshadow is calculated within 

  SOLAR_AZIMUTH_ANGLE = ee.Number(image.get('SUN_AZIMUTH'))
  SOLAR_ELEVATION =ee.Number(image.get('SUN_ELEVATION'))
  
  hillShade = ee.Terrain.hillshade(MergedDEM, SOLAR_AZIMUTH_ANGLE,
  SOLAR_ELEVATION).rename(['hillShade'])
               
  return hillShade
  
def CalcHillShadows(image, geo):
  MergedDEM = ee.Image("users/eeProject/MERIT").clip(geo.buffer(3000)) # Area buffered that hillshadow is calculated within 
  SOLAR_AZIMUTH_ANGLE = ee.Number(image.get('SUN_AZIMUTH'))
  SOLAR_ZENITH_ANGLE =ee.Number(90).subtract(image.get('SUN_ELEVATION'))

  hillShadow = ee.Terrain.hillShadow(MergedDEM, SOLAR_AZIMUTH_ANGLE,SOLAR_ZENITH_ANGLE, 30).rename(['hillShadow'])
    
  return hillShadow



# Define channel function 

### This function extracts pixel values along the nhd centerline within the veroni polygons. Without this, we would end up pulling in water pixels outside of the river reaches (pools, tributaries, etc.)


In [14]:
def ExtractChannel(image, centerline, bound, maxDistance):
  cost = image.Not().cumulativeCost(ee.Image().toByte().paint(centerline, 1).And(image), maxDistance, False)
  channelMask = cost.eq(0).unmask(0).clip(bound).rename(['channelMask'])
  channel = image.mask(channelMask).unmask(0)
  return channel

## These function do the following: 

* Remove geometry field

* Add bands with negative pixel values (band value < 0), these will be used later to count the number of negative pixels within a site location 


In [15]:
def removeGeo(i):
    return i.setGeometry(None)

def add_negative_Aerosol(image):
      
      Aerosol = image.select('Aerosol')
      negative_Aerosol = Aerosol.lt(0).rename('negative_Aerosol')
  
      return(negative_Aerosol)


def add_negative_Red(image):

      Red = image.select('Red')
      negative_Red = Red.lt(0).rename('negative_Red')
  
      return(negative_Red)

def add_negative_Blue(image):

      Blue = image.select('Blue')
      negative_Blue = Blue.lt(0).rename('negative_Blue')
  
      return(negative_Blue)

def add_negative_Green(image):

      Green = image.select('Green')
      negative_Green = Green.lt(0).rename('negative_Green')
  
      return(negative_Green)


def add_negative_Nir(image):

      Nir = image.select('Nir')
      negative_Nir = Nir.lt(0).rename('negative_Nir')
  
      return(negative_Nir)

def add_negative_Swir1(image):

      Swir1 = image.select('Swir1')
      negative_Swir1 = Swir1.lt(0).rename('negative_Swir1')
  
      return(negative_Swir1)

def add_negative_Swir2(image):

      Swir2 = image.select('Swir2')
      negative_Swir2 = Swir2.lt(0).rename('negative_Swir2')
  
      return(negative_Swir2)

# Surface reflectance pull function

* The pull function first defines variables that will be used to mask each image (i.e. clouds, hillshadows, dswe). 

* Then a waterOut variable is defined as the image with all of the masks applied and additional variables to be exported within our table (i.e. the DSWE value, the cloud value, etc.)

* Then the combined reducer calculates medians and standard deviations for our band values, counts negative pixel values and pixels where dswe == 1 or the algal mask threshold is satisfied, gets mean hillshdaow and cloud values, and collects the first non null value for various image metadata properties

* The extract channel function is then called to apply our pull function over the channel mask

In [16]:
def pull_57(image):
 
    r = add_rad_mask(image).select('radsat')
    f = AddFmask(image).select('fmask')
    clouds = f.gte(2).rename('clouds')
    algal_mask = Dswe(image).select('algal_mask')
    a = sr_aerosol(image).select('medHighAero')
    hs = CalcHillShadows(image, reach_polygon.geometry()).select('hillShadow')
    negative_Aerosol = add_negative_Aerosol(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1)) 
    negative_Red = add_negative_Red(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Blue = add_negative_Blue(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Green = add_negative_Green(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Nir = add_negative_Nir(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Swir1 = add_negative_Swir1(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Swir2 = add_negative_Swir2(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    dummy = (image.select(['Blue'],['algal_mask']).updateMask(clouds.eq(0)).updateMask(algal_mask.eq(1)).updateMask(hs.eq(1)).updateMask(r.eq(1)))
    hs0 = hs.eq(0).rename('shadow').selfMask().updateMask(clouds.eq(0)).updateMask(algal_mask.eq(1)).updateMask(r.eq(1)) 
    cover = image.metadata('CLOUD_COVER')
    z = image.metadata('SUN_ELEVATION')
    a = image.metadata('SUN_AZIMUTH')
    date_number = ee.Number(image.get("system:time_start"))
    date = image.constant(date_number).rename("date")
    image_quality_number = ee.Number(image.get("IMAGE_QUALITY"))
    image_quality = image.constant(image_quality_number).rename("IMAGE_QUALITY")
    pixOut = (image.addBands(algal_mask)
              .addBands(image.select(['Aerosol'],['sd_Aerosol']))
              .addBands(image.select(['Blue'],['sd_Blue']))
              .addBands(image.select(['Green'],['sd_Green']))
              .addBands(image.select(['Red'],['sd_Red']))
              .addBands(image.select(['Nir'],['sd_Nir']))
              .addBands(image.select(['Swir1'],['sd_Swir1']))
              .addBands(image.select(['Swir2'],['sd_Swir2']))
              .updateMask(algal_mask.eq(1))
              .updateMask(clouds.eq(0))
              .updateMask(r.eq(1))
              .updateMask(hs.eq(1))
              .addBands(negative_Aerosol)
              .addBands(negative_Red)
              .addBands(negative_Blue)
              .addBands(negative_Green)
              .addBands(negative_Nir)
              .addBands(negative_Swir1)
              .addBands(negative_Swir2)
              .addBands(dummy)
              .addBands(hs0)
              .addBands(hs)
              .addBands(clouds)
              .addBands(cover)
              .addBands(z)
              .addBands(a)
              .addBands(date)
              .addBands(image_quality)
              .addBands(r))

    combinedReducer = (ee.Reducer.median().unweighted()
    .forEachBand(pixOut.select(['Aerosol', 'Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2','Surface_temp_kelvin', 'pixel_qa', 'radsat_qa', 'algal_mask']))
    .combine(ee.Reducer.stdDev().unweighted().forEachBand(pixOut.select(['sd_Aerosol','sd_Blue', 'sd_Green', 'sd_Red','sd_Nir', 'sd_Swir1','sd_Swir2'])), '', False)
    .combine(ee.Reducer.count().unweighted().forEachBand(pixOut.select(['negative_Aerosol', 'negative_Blue', 'negative_Green', 'negative_Red', 'negative_Nir', 'negative_Swir1', 'negative_Swir2','algal_mask','shadow' ])), 'pCount_', False)
    .combine(ee.Reducer.mean().unweighted().forEachBand(pixOut.select(['hillShadow', 'clouds'])), '', False)
    .combine(ee.Reducer.firstNonNull().unweighted().forEachBand(pixOut.select(['CLOUD_COVER', 'SUN_ELEVATION', 'SUN_AZIMUTH', 'date', 'IMAGE_QUALITY', 'radsat']))))
    
    
    pixChannel = ExtractChannel(pixOut.select('algal_mask').eq(1), reach_centerline, reach_polygon.geometry(), 500)
    pixOut2 = pixOut.updateMask(pixChannel)
    lsout = pixOut2.reduceRegions(reach_polygon, combinedReducer, 30)

    out = lsout.map(removeGeo)
    
    return out

In [17]:
def pull_89(image):
 
    r = add_rad_mask(image).select('radsat')
    f = AddFmask(image).select('fmask')
    a = sr_aerosol(image).select('medHighAero')
    clouds = f.gte(2).rename('clouds')
    algal_mask = Dswe(image).select('algal_mask')
    a = sr_aerosol(image).select('medHighAero')
    hs = CalcHillShadows(image, reach_polygon.geometry()).select('hillShadow')
    negative_Aerosol = add_negative_Aerosol(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1)) 
    negative_Red = add_negative_Red(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Blue = add_negative_Blue(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Green = add_negative_Green(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Nir = add_negative_Nir(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Swir1 = add_negative_Swir1(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    negative_Swir2 = add_negative_Swir2(image).selfMask().updateMask(algal_mask.eq(1)).updateMask(clouds.eq(0)).updateMask(hs.eq(1)).updateMask(r.eq(1))  
    dummy = (image.select(['Blue'],['algal_mask']).updateMask(clouds.eq(0)).updateMask(algal_mask.eq(1)).updateMask(hs.eq(1)).updateMask(r.eq(1)))
    hs0 = hs.eq(0).rename('shadow').selfMask().updateMask(clouds.eq(0)).updateMask(algal_mask.eq(1)).updateMask(r.eq(1)) 
    cover = image.metadata('CLOUD_COVER')
    z = image.metadata('SUN_ELEVATION')
    a = image.metadata('SUN_AZIMUTH')
    date_number = ee.Number(image.get("system:time_start"))
    date = image.constant(date_number).rename("date")
    image_quality_number = ee.Number(image.get("IMAGE_QUALITY"))
    image_quality = image.constant(image_quality_number).rename("IMAGE_QUALITY")
    pixOut = (image.addBands(algal_mask)
              .addBands(image.select(['Aerosol'],['sd_Aerosol']))
              .addBands(image.select(['Blue'],['sd_Blue']))
              .addBands(image.select(['Green'],['sd_Green']))
              .addBands(image.select(['Red'],['sd_Red']))
              .addBands(image.select(['Nir'],['sd_Nir']))
              .addBands(image.select(['Swir1'],['sd_Swir1']))
              .addBands(image.select(['Swir2'],['sd_Swir2']))
              .updateMask(algal_mask.eq(1))
              .updateMask(clouds.eq(0))
              .updateMask(r.eq(1))
              .updateMask(hs.eq(1))
              .addBands(negative_Aerosol)
              .addBands(negative_Red)
              .addBands(negative_Blue)
              .addBands(negative_Green)
              .addBands(negative_Nir)
              .addBands(negative_Swir1)
              .addBands(negative_Swir2)
              .addBands(dummy)
              .addBands(hs0)
              .addBands(hs)
              .addBands(clouds)
              .addBands(cover)
              .addBands(z)
              .addBands(a)
              .addBands(date)
              .addBands(image_quality)
              .addBands(r))

    combinedReducer = (ee.Reducer.median().unweighted()
    .forEachBand(pixOut.select(['Aerosol', 'Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2','Surface_temp_kelvin', 'pixel_qa', 'algal_mask']))
    .combine(ee.Reducer.stdDev().unweighted().forEachBand(pixOut.select(['sd_Aerosol','sd_Blue', 'sd_Green', 'sd_Red','sd_Nir', 'sd_Swir1','sd_Swir2'])), '', False)
    .combine(ee.Reducer.count().unweighted().forEachBand(pixOut.select(['negative_Aerosol', 'negative_Blue', 'negative_Green', 'negative_Red', 'negative_Nir', 'negative_Swir1', 'negative_Swir2','algal_mask','shadow' ])), 'pCount_', False)
    .combine(ee.Reducer.mean().unweighted().forEachBand(pixOut.select(['hillShadow', 'clouds'])), '', False)
    .combine(ee.Reducer.firstNonNull().unweighted().forEachBand(pixOut.select(['CLOUD_COVER', 'SUN_ELEVATION', 'SUN_AZIMUTH', 'date', 'IMAGE_QUALITY', 'radsat']))))
    
    
    pixChannel = ExtractChannel(pixOut.select('algal_mask').eq(1), reach_centerline, reach_polygon.geometry(), 500)
    pixOut2 = pixOut.updateMask(pixChannel)
    lsout = pixOut2.reduceRegions(reach_polygon, combinedReducer, 30)

    out = lsout.map(removeGeo)
    
    return out

Function for limiting number of tasks in Task Manager

In [18]:
def maximum_no_of_tasks(MaxNActive, waitingPeriod):
  ##maintain a maximum number of active tasks
  time.sleep(10)
  ## initialize submitting jobs
  ts = list(ee.batch.Task.list())

  NActive = 0
  for task in ts:
       if ('RUNNING' in str(task) or 'READY' in str(task)):
           NActive += 1
  ## wait if the number of current active tasks reach the maximum number
  ## defined in MaxNActive
  while (NActive >= MaxNActive):
      time.sleep(waitingPeriod) # if reach or over maximum no. of active tasks, wait for 2min and check again
      ts = list(ee.batch.Task.list())
      NActive = 0
      for task in ts:
        if ('RUNNING' in str(task) or 'READY' in str(task)):
          NActive += 1
  return()

Merging landsat collections and scaling band values

In [37]:
# LS Collection 2 has a different scaling parameter that needs to be applied to the optical bands. Also, thermal bands need scale factors and these are different for LS8 and LS5/LS7

# Clip image function
def clipImage(image):
  return image.clip(reach_polygon.geometry())

def scale_ls5_ls7(image):

  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

  return image.addBands(opticalBands, overwrite = True)\
  .addBands(thermalBand,  overwrite = True)

def scale_ls8_ls9(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0)

  return image.addBands(opticalBands, overwrite = True)\
  .addBands(thermalBand,  overwrite = True)


def rename_image_quality(i):
    return i.set({"IMAGE_QUALITY": i.get("IMAGE_QUALITY_OLI")})

# Aerosol doesn't exist for LS 5 and LS 7 so we will have to add a dummy band with fill value of -99 for those collections
dummyAerosol = ee.Image(-99).rename('Null_CS')

def add_dummy_Aerosol(i):
    return i.addBands(dummyAerosol)

l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
  .map(rename_image_quality)
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
  .map(rename_image_quality)
l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
  .map(add_dummy_Aerosol)
l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')\
  .map(add_dummy_Aerosol)

ls9_bands = ['SR_B1', 'SR_B2','SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7', 'ST_B10', 'QA_PIXEL']
ls8_bands = ['SR_B1','SR_B2','SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7', 'ST_B10','QA_PIXEL']
ls7_bands = ['Null_CS','SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'ST_B6', 'QA_PIXEL', 'QA_RADSAT']
ls5_bands = ['Null_CS','SR_B1','SR_B2','SR_B3', 'SR_B4', 'SR_B5','SR_B7', 'ST_B6','QA_PIXEL', 'QA_RADSAT']
bands = ['Aerosol','Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'Surface_temp_kelvin', 'pixel_qa', 'radsat_qa']

ls_collection57 = l7.merge(l5).filter(ee.Filter.lt('CLOUD_COVER', 50)).filterDate('1984-03-16', '2022-11-01')


ls9 = l9.map(scale_ls8_ls9).select(ls9_bands, bands)
ls8 = l8.map(scale_ls8_ls9).select(ls8_bands, bands)
ls7 = l7.map(scale_ls5_ls7).select(ls7_bands, bands)
ls5 = l5.map(scale_ls5_ls7).select(ls5_bands, bands)


ls_collection57 = (ls_collection57
.map(apply_fill_mask)
.map(scale_ls5_ls7)
.map(apply_realistic_mask)
.map(apply_opac_mask))

ls_collection57 = ls_collection57.select(ls5_bands, bands)

# Filtering for cloud cover less than 50 reduces processing time (don't know if this should be higher or lower)
#ls_collection57 = ls7.merge(ls5).filter(ee.Filter.lt('CLOUD_COVER', 50)).filterDate('1984-03-16', '2022-11-01')

#ls_collection89 = ls9.merge(ls8).filter(ee.Filter.lt('CLOUD_COVER', 50)).filterDate('1984-03-16', '2022-11-01')


Preparing to loop over site identifiers

In [38]:

nhd_polygons = ee.FeatureCollection("projects/ee-samsillen0/assets/nhd_polygons_fixed")
# Load nhd centerlines
nhd_centerlines = ee.FeatureCollection("projects/ee-samsillen0/assets/nhd_centerlines_fixed")
wrs = ee.FeatureCollection("projects/ee-samsillen0/assets/wrs2_asc_desc").filterMetadata('MODE', 'equals', 'D')

# This pr csv has the tendency to be finnicky ; due to the leading zeros in the pr column
pr_shp = ee.FeatureCollection("projects/ee-samsillen0/assets/allwqplagostiles").filterBounds(nhd_centerlines)


In [40]:
lakesort = pr_shp.sort('PR')

lakeID = lakesort.aggregate_array('PR').getInfo() 

# Make a folder in your google drive manually to output data , then we can use this loop in case we have to start over and we won't end up starting from 0

dlDir = "G:\My Drive\lc02_full_add_masks_v2"
filesDown = os.listdir(dlDir) 
filesDown = [str(i.replace(".csv", "")) for i in filesDown]

lakeID  = [i for i in lakeID if i not in filesDown]

print(len(lakeID))
#len(lakeID))

for x in range(0, len(lakeID)):
        
    print(lakeID[x])
    
    tile = wrs.filterMetadata('PR', 'equals', lakeID[x])
    
    reach_polygon = nhd_polygons.filterBounds(tile.geometry())
    
    reach_centerline = nhd_centerlines.filterBounds(tile.geometry())

    stack = ls_collection57.filterBounds(tile.geometry().centroid())
    
    out = stack.map(pull_57).flatten() 
    out = out.filter(ee.Filter.notNull(['Blue']))

    dataOut = ee.batch.Export.table.toDrive(collection = out,\
                                            description = str(lakeID[x]),\
                                            folder = 'lc02_57_add_masks_v2',\
                                            fileFormat = 'csv',\
                                            selectors  = ['Aerosol', 'sd_Aerosol', 'pCount_negative_Aerosol', 'Blue','sd_Blue','pCount_negative_Blue','Green','sd_Green', 'pCount_negative_Green', 'Red','sd_Red', 'pCount_negative_Red','Nir', 'sd_Nir','pCount_negative_Nir','Swir1','sd_Swir1','pCount_negative_Swir1','Swir2','sd_Swir2','pCount_negative_Swir2','Surface_temp_kelvin','pixel_qa','clouds','algal_mask','hillShadow','pCount_algal_mask','pCount_shadow','system:index', 'IMAGE_QUALITY','date','CLOUD_COVER','SUN_ELEVATION','SUN_AZIMUTH','COMID', 'radsat'])    

  # Check how many existing tasks are running and take a break if it's >15  
    maximum_no_of_tasks(15, 60)
  # Send next task.
    dataOut.start()

# Make sure all Earth engine tasks are completed prior to moving on.  
maximum_no_of_tasks(1,300)
print('done')


372
010028
010029
011027
011028
011029
011030
012027
012028
012029
012030
012031
013028
013029
013030
013031
013032
014029
014030


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))